##import library


In [1]:
!pip install PyPDF2==3.0.1 transformers accelerate torch huggingface_hub
import re
import pandas as pd
import requests
import torch
from PyPDF2 import PdfReader
from google.colab import files
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import BytesIO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

#passing to llm


In [2]:
notebook_login()

In [3]:
model_id = "Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:999: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

##upload files

In [4]:
uploaded = files.upload()
csv_path = list(uploaded.keys())[0]
print("Dokumen :", csv_path)
df = pd.read_csv(csv_path)

Saving splp_perkara_202504150803.csv to splp_perkara_202504150803 (1).csv
Dokumen : splp_perkara_202504150803 (1).csv


In [5]:
df = df.head(10)
df

,id,putusancode,no_perkara,pokok_perkara,pemohon,amar_putusan,download_count,supporting_file,date,date_dpr,keywords,status,link_hit,status_dpr,user_input,tanggal_input,user_update,tanggal_update
0,1,4581,46/PUU-XXII/2024,Pengujian Materiil Undang-Undang Nomor 10 Tahu...,"Sahbirin Noor, S.Sos, M.H. (Pemohon I); Ahmad ...",Dalam Provisi: Menolak permohonan provisi Pemo...,1045,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T13:14:00+07:00,2024-07-30 13:14:00,masa jabatan kepala daerah pilkada serentak 2024,Menolak Sebagian (Tidak Dapat Diterima),https://api-test.mkri.id/putusan/download/4581,1,ibnu.hisyam,2024-08-07 10:30:11,ibnu.hisyam,2024-08-07 10:30:11
1,2,4582,45/PUU-XXII/2024,Pengujian Materiil Undang-Undang Nomor 7 Tahun...,Ir. H. Didi Apriadi,Menolak permohonan Pemohon untuk seluruhnya.,939,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T12:48:00+07:00,2024-07-30 12:48:00,"disproporsionalitas hasil pemilu, Pasal 414 ay...",Menolak Seluruhnya,https://api-test.mkri.id/putusan/download/4582,1,ibnu.hisyam,2024-08-07 10:30:46,ibnu.hisyam,2024-08-07 10:30:46
2,3,4583,39/PUU-XXII/2024,Pengujian Materiil Undang-Undang Nomor 19 Tahu...,Muhammad Kahfi Andhika Bayu Adji,Menyatakan permohonan Pemohon tidak dapat dite...,259,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T12:35:00+07:00,2024-07-30 12:35:00,"kerugian minimal dalam korupsi, 1 milyar, koru...",Tidak Dapat Diterima,https://api-test.mkri.id/putusan/download/4583,1,ibnu.hisyam,2024-08-07 10:31:23,ibnu.hisyam,2024-08-07 10:31:23
3,4,4584,35/PUU-XXII/2024,Pengujian Materiil Undang-Undang Nomor 13 Tahu...,"Leonardo Olefins Hamonangan., S.H",Menolak permohonan Pemohon untuk seluruhnya.,609,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T12:26:00+07:00,2024-07-30 12:26:00,"batas usia, diskriminasi, ketenagakerjaan, low...",Menolak Seluruhnya,https://api-test.mkri.id/putusan/download/4584,1,ibnu.hisyam,2024-08-07 11:32:47,ibnu.hisyam,2024-08-07 11:32:47
4,5,4585,34/PUU-XXII/2024,Pengujian Materiil Kitab Undang- Undang Hukum ...,Sunyoto (Pemohon I) dan Jaka Fiton (Pemohon II),Menyatakan permohonan para Pemohon tidak dapat...,383,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T12:03:00+07:00,2024-07-30 12:03:00,"batas usia pensiun notaris, pembatasan wilayah...",Tidak Dapat Diterima,https://api-test.mkri.id/putusan/download/4585,1,ibnu.hisyam,2024-08-07 12:08:50,ibnu.hisyam,2024-08-07 12:08:50
5,6,4586,162/PUU-XXI/2023,Pengujian Materiil Undang-Undang Nomor 20 Tahu...,Djunatan Prambudi,Menolak permohonan Pemohon untuk seluruhnya.,291,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T11:48:00+07:00,2024-07-30 11:48:00,"syarat merek, kriteria pembeda merek",Menolak Seluruhnya,https://api-test.mkri.id/putusan/download/4586,1,ibnu.hisyam,2024-08-07 12:08:50,ibnu.hisyam,2024-08-07 12:08:50
6,7,4587,144/PUU-XXI/2023,Pengujian Materil Undang-Undang Nomor 20 Tahun...,Ricky Thio,1. Mengabulkan permohonan Pemohon untuk sebagi...,333,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T11:24:00+07:00,2024-07-30 11:24:00,"penghapusan merek, yang diajukan oleh pihak ke...",Mengabulkan Sebagian (Ditolak),https://api-test.mkri.id/putusan/download/4587,1,ibnu.hisyam,2024-08-07 12:10:21,ibnu.hisyam,2024-08-07 12:10:21
7,8,4588,57/PUU-XXII/2024,Pengujian Materiil Undang Undang Nomor 7 Tahun...,Pasai,Menyatakan permohonan Pemohon gugur.,278,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T10:42:00+07:00,2024-07-30 10:42:00,"ambang batas, pemilihan pimpinan DPR",Gugur,https://api-test.mkri.id/putusan/download/4588,1,ibnu.hisyam,2024-08-07 12:10:21,ibnu.hisyam,2024-08-07 12:10:21
8,9,4589,53/PUU-XXII/2024,Pengujian Undang-Undang Nomor 1 Tahun 1970 ten...,"Dr. Charles Mangaraja Tampubolon, S.H., M.K.K.K.",Menyatakan permohonan Pemohon gugur.,281,https://s.mkri.id/public/content/persidangan/p...,2024-07-30T10:36:00+07:00,2024-07-30 10:36:00,"sanksi pidana, keselamatan kerja",Gugur,https://api-test.mkri.id/put

##Function process

In [8]:
komisi_list = """
Berikut adalah daftar 13 Komisi DPR RI:
1. Komisi I: Pertahanan, luar negeri, intelijen, komunikasi dan informatika.
2. Komisi II: Pemerintahan dalam negeri, aparatur negara, dan reformasi birokrasi.
3. Komisi III: Hukum, HAM, dan keamanan.
4. Komisi IV: Pertanian, lingkungan hidup, kehutanan dan kelautan.
5. Komisi V: Infrastruktur dan transportasi.
6. Komisi VI: Perdagangan, koperasi, UMKM, dan BUMN.
7. Komisi VII: Energi, riset dan teknologi, dan lingkungan hidup.
8. Komisi VIII: Sosial, agama, dan pemberdayaan perempuan.
9. Komisi IX: Kesehatan, ketenagakerjaan, dan transmigrasi.
10. Komisi X: Pendidikan, olahraga, pariwisata, dan kebudayaan.
11. Komisi XI: Keuangan dan perbankan.
12. Komisi XII: Ekonomi kreatif dan digital.
13. Komisi XIII: Komunikasi, penyiaran dan literasi digital.
"""

komisi_map = {
    "Komisi I": "Polkam",
    "Komisi II": "Polkam",
    "Komisi III": "Polkam",
    "Komisi XI": "EKKU",
    "Komisi XII": "EKKU",
    "Komisi XIII": "EKKU",
    "Komisi IV": "Korinbang",
    "Komisi V": "Korinbang",
    "Komisi VI": "Korinbang",
    "Komisi VII": "Korinbang",
    "Komisi VIII": "Kesra",
    "Komisi IX": "Kesra",
    "Komisi X": "Kesra"
}

In [11]:
def extract_text_from_pdf_bytes(pdf_path):
  reader = PdfReader(BytesIO(pdf_path))
  full_text = ""
  for page in reader.pages:
    full_text += page.extract_text()+"\n"
  return full_text

def map_status_to_group(status):
    status = status.lower()
    if any(s in status for s in ["mengabulkan seluruhnya", "mengabulkan sebagian", "dikabulkan sebagian"]):
        return "Putusan Dikabulkan"
    elif any(s in status for s in ["menolak seluruhnya", "ditolak"]):
        return "Putusan Ditolak"
    elif "tidak dapat diterima" in status:
        return "Putusan Tidak Dapat Diterima"
    elif any(s in status for s in ["gugur ", "ditarik kembali"]):
        return "Putusan Gugur"
    elif any(s in status for s in ["mengabulkan sebagian (ditolak)", "mengabulkan sebagian (tidak dapat diterima)","mengabulkan sebagian (ditolak dan tidak dapat diterima)", "mengabulkan sebagian (ditolak, tidak dapat diterima dan tidak berwenang)","mengabulkan sebagian (ditolak, tidak dapat diterima, tidak berwenang dan gugur)", "menolak sebagian (tidak dapat diterima)", "menolak sebagian (tidak dapat diterima dan gugur)", "menolak sebagian (tidak dapat diterima dan tidak berwenang)" ]):
        return "Putusan Campuran"
    elif "tidak berwenang" in status:
        return "Putusan Tidak Berwenang"
    elif any(s in status for s in ["sela", "sela / provisi"]):
        return "Putusan Sela / Provisi"
    else:
        return "Status Tidak Valid / Belum Diisi"

def extract_petitum_section(text):
  text_cleaned = re.sub(r'\s+', ' ', text, flags=re.MULTILINE).lower()

  match = re.search(r'(petitum.*?)pertimbangan hukum', text_cleaned, re.DOTALL)
  if match:
    petitum_text = match.group(1).strip()
    return petitum_text
  return None

def extract_amar_putusan_section(text):
    text_cleaned = re.sub(r'\s+', ' ', text, flags=re.MULTILINE).lower()
    match = re.search(r'5\.\s*Amar Putusan(.*?)(?=6\.\s*pendapat berbeda|ttd)', text_cleaned, re.DOTALL | re.IGNORECASE)

    if match:
      amar_text = match.group(1).strip()
      return amar_text
    return None

def generate_prompt_petitum(petitum_text, komisi_list, keywords):
    return f"""
Analisis teks tsi seuntutan berikut secara cermat, lalu identifikasi dan klasifikasikan isu-isu utama dalam tuntutan tersebut. Tentukan komisi DPR RI yang paling relevan untuk menangani permasalahan tersebut, berdasarkan kesesuaian dengan tugas dan ruang lingkup masing-masing komisi.
berikan jawaban berupa "Komisi XXX : [alasan relevansi klasifiksi secara singkat]".
{komisi_list}

teks tuntutan yang akan di analisis:
{petitum_text}

Berikut kata kunci utama yang dapat membantu analisis:
{keywords}

Klasifikasi harus berdasarkan substansi permasalahan dalam tuntutan, bukan sekadar kata kunci umum. Pastikan bahwa relevansi dengan komisi yang dipilih benar-benar sesuai dengan fungsi dan kewenangan yang diatur dalam tugas dan ruang lingkup tiap komisi DPR RI.

"""

def generate_prompt_amar(amar_text, komisi_list):
    return f"""
Anda adalah seorang ahli hukum yang berpengalaman dalam menganalisis dan mengklasifikasikan dokumen hukum, khususnya putusan pengadilan. Anda memiliki pemahaman mendalam mengenai tugas dan ruang lingkup dari 13 komisi di DPR RI, serta kemampuan untuk menilai relevansi konteks permasalahan dalam teks hukum.
Tugas Anda adalah mengklasifikasikan sebuah teks putusan pengadilan ke dalam salah satu dari 13 komisi DPR RI.
Teks putusan pengadilan:
{amar_text}

{komisi_list}

Saat mengklasifikasikan, harap pertimbangkan konteks permasalahan yang terdapat dalam teks dan bagaimana hal itu terkait dengan tugas dan ruang lingkup masing-masing komisi. Pastikan untuk memberikan alasan singkat mengenai pilihan Anda.


"""

def determine_bidang_koordinator(hasil_klasifikasi):
    match = re.search(r"\bkomisi\s+(i{1,3}|iv|v|vi{0,3}|ix|x|xi|xii|xiii)\b", hasil_klasifikasi, re.IGNORECASE)
    if match:
        romawi = match.group(1).upper()
        nama_komisi = f"Komisi {romawi}"
        return komisi_map.get(nama_komisi, hasil_klasifikasi)
    return hasil_klasifikasi

def classify_text_with_llm(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    input_length = inputs['input_ids'].shape[1]
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.2,
            do_sample=False,
        )
    generated_tokens = outputs[0][input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    match = re.search(r'\bkomisi\s+(?:[ivx]+)', generated_text, re.IGNORECASE)
    return match.group(0).strip() if match else generated_text.strip()


In [12]:
komisi_list_result = []
bidang_koordinator_result = []


for i, row in df.iterrows():
    try:
        print(f"Memproses dokumen ke-{i+1}")
        status = row["status"]
        print(status)
        status_group = map_status_to_group(status)
        print(status_group)
        url = row["supporting_file"]
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception("Gagal download PDF.")
        pdf_text = extract_text_from_pdf_bytes(response.content)

        if status_group in ["Putusan Dikabulkan", "Putusan Campuran", "Putusan Sela / Provisi"]:
            amar_text = extract_amar_putusan_section(pdf_text)
            if not amar_text:
                hasil_klasifikasi = status
            else:
                prompt = generate_prompt_amar(amar_text, komisi_list)
                hasil_klasifikasi = classify_text_with_llm(prompt)

        elif status_group in ["Putusan Ditolak", "Putusan Tidak Dapat Diterima"]:
            petitum_text = extract_petitum_section(pdf_text)
            if not petitum_text:
                hasil_klasifikasi = "Permohonan gugur"
            else:
                keywords = row["keywords"]
                prompt = generate_prompt_petitum(petitum_text, komisi_list, keywords)
                hasil_klasifikasi = classify_text_with_llm(prompt)

        elif status_group in ["Putusan Gugur", "Putusan Tidak Berwenang", "Status Tidak Valid / Belum Diisi"]:
            hasil_klasifikasi = status

        else:
            hasil_klasifikasi = f"Status tidak dikenali: {status}"

        komisi_list_result.append(hasil_klasifikasi)

        # Gunakan hasil klasifikasi langsung untuk bidang koordinator
        bidang_koordinator = determine_bidang_koordinator(hasil_klasifikasi)
        bidang_koordinator_result.append(bidang_koordinator)

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        komisi_list_result.append(error_msg)
        bidang_koordinator_result.append(error_msg)

# Tambahkan hasil ke DataFrame
df["komisi"] = komisi_list_result
df["bidang_koordinator"] = bidang_koordinator_result

# Simpan ke file
df.to_csv("hasil_klasifikasi_petitum.csv", index=False)
print("✅ Proses selesai. File disimpan: hasil_klasifikasi_petitum.csv")

Memproses dokumen ke-1
Menolak Sebagian (Tidak Dapat Diterima)
Putusan Tidak Dapat Diterima


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-2
Menolak Seluruhnya
Putusan Ditolak


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-3
Tidak Dapat Diterima
Putusan Tidak Dapat Diterima


...

Memproses dokumen ke-6
Menolak Seluruhnya
Putusan Ditolak


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-7
Mengabulkan Sebagian (Ditolak)
Putusan Dikabulkan


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Memproses dokumen ke-8
Gugur
Status Tidak Valid / Belum Diisi
Memproses dokumen ke-9
Gugur
Status Tidak Valid / Belum Diisi
Memproses dokumen ke-10
Gugur
Status Tidak Valid / Belum Diisi
✅ Proses selesai. File disimpan: hasil_klasifikasi_petitum.csv
